This code is the general case for the problem found in 5.3.4 of the textbook Power Flow Analysis by N. V. Ramana

**Imports**

In [1]:
import numpy as np
from numpy.linalg import inv

**Initial Values**

In [2]:
# The bus data
# For each bus the data is as follows:
# 0:BusNum, 1:PG, 2:QG, 3:PD, 4:QD, 5:V, 6:PA, 7:BusType
# Where BusType 1 is Slack, 2 is PV, and 3 is PQ
Buses = np.array([[1, 0, 0, -0.9, -0.4, 1.02, 0, 1], [2, 1.4, 0, 0, 0, 1.03, 0, 2], [3, 0, 0, -1.1, -0.4, 0, 0, 3]])

# Load information on P, Q, V, and PA into variables
P = []
Q = []
V = []
PA = []
for i in Buses:
    V.append(i[5])
    PA.append(i[6])
    if i[7] == 2:
        P.append(i[1])
        Q.append(i[2])
    else:
        P.append(i[3])
        Q.append(i[4])

P = np.array(P)
Q = np.array(Q)
V = np.array(V)
PA = np.array(PA)        
display(P)
display(Q)
display(V)
display(PA)

array([-0.9,  1.4, -1.1])

array([-0.4,  0. , -0.4])

array([1.02, 1.03, 0.  ])

array([0., 0., 0.])

In [3]:
# The series impedance of each line
# Note that Buses.shape[0] is the number of rows in Buses (ie. the number of buses)
SI = np.empty(Buses.shape[0], dtype = complex)
SI.fill(complex(0.025, 0.1))

**Form J, Y<sub>bus</sub>, G, and B**

In [4]:
SA = 1/SI
Y = np.zeros((Buses.shape[0], Buses.shape[0]),  dtype = complex)
for i in range(Y.shape[0]):
    for j in range (Y.shape[1]):
        if i != j:
            Y[i, j] = Y[i, j] - SA[i]
            Y[i, i] = Y[i, i] + SA[i]
            
G = Y.real
B = Y.imag
display(Y)
display(G)
display(B)

array([[ 4.70588235-18.82352941j, -2.35294118 +9.41176471j,
        -2.35294118 +9.41176471j],
       [-2.35294118 +9.41176471j,  4.70588235-18.82352941j,
        -2.35294118 +9.41176471j],
       [-2.35294118 +9.41176471j, -2.35294118 +9.41176471j,
         4.70588235-18.82352941j]])

array([[ 4.70588235, -2.35294118, -2.35294118],
       [-2.35294118,  4.70588235, -2.35294118],
       [-2.35294118, -2.35294118,  4.70588235]])

array([[-18.82352941,   9.41176471,   9.41176471],
       [  9.41176471, -18.82352941,   9.41176471],
       [  9.41176471,   9.41176471, -18.82352941]])

**Compute Powers**

In [5]:
# Flat start for bus voltages
# Unknown values for bus volage will be set to 1
for i in range(V.size):
    if V[i] == 0:
        V[i] = 1
        
V

array([1.02, 1.03, 1.  ])

In [6]:
P_new = np.zeros(Buses.shape[0])
Q_new = np.zeros(Buses.shape[0])
# We don't need to calculate for the slack bus (the first bus), so we start at the second bus
for i in range(1, Buses.shape[0]):
    for j in range (Buses.shape[0]):
        P_new[i] = P_new[i]+V[i]*V[j]*(G[i][j]*np.cos(PA[i]-PA[j])+B[i][j]*np.sin(PA[i]-PA[j]))
        Q_new[i] = Q_new[i]+V[i]*V[j]*(G[i][j]*np.sin(PA[i]-PA[j])-B[i][j]*np.cos(PA[i]-PA[j]))
        
display(P_new)
display(Q_new)

array([ 0.        ,  0.09694118, -0.11764706])

array([ 0.        ,  0.38776471, -0.47058824])

**Compute Power Mismatches**

In [7]:
Delta_P = []
Delta_Q = []
for i in range(1, Buses.shape[0]):
    # Buses[i][7] is Bus i's BusType, 2 is PV
    if Buses[i][7] == 2:
        Delta_P.append(P[i] - P_new[i])
    # else (PQ)
    else:
        Delta_P.append(P[i] - P_new[i])
        Delta_Q.append(Q[i] - Q_new[i])
        
P = P_new
Q = Q_new
Delta_PQ = np.append(Delta_P, Delta_Q)
Delta_PQ

array([ 1.30305882, -0.98235294,  0.07058824])

**Compute Jacobian Elements**

In [8]:
H = np.zeros((Buses.shape[0]-1, Buses.shape[0]-1))
N = np.zeros((Buses.shape[0]-1, Buses.shape[0]-1))
J = np.zeros((Buses.shape[0]-1, Buses.shape[0]-1))
L = np.zeros((Buses.shape[0]-1, Buses.shape[0]-1))
for i in range(1, Buses.shape[0]):
    for j in range(1, Buses.shape[0]):
        if i == j:
            H[i-1][j-1] = -Q[i]-B[i][i]*(V[i]**2)
            # Buses[i][7] is Bus i's BusType, 2 is PV
            if Buses[i][7] != 2:
                N[i-1][j-1] = P[i]+G[i][i]*(V[i]**2)
                J[i-1][j-1] = P[i]-G[i][i]*(V[i]**2)
                L[i-1][j-1] = Q[i]-B[i][i]*(V[i]**2)
        else:
            H[i-1][j-1] = V[i]*V[j]*(G[i][j]*np.cos(PA[i]-PA[j])+B[i][j]*np.sin(PA[i]-PA[j]))
            if Buses[j][7] != 2:
                N[i-1][j-1] = V[j]*V[i]*(G[i][j]*np.sin(PA[i]-PA[j])+B[i][j]*np.cos(PA[i]-PA[j]))
            if Buses[i][7] != 2:
                J[i-1][j-1] = -V[i]*V[j]*(G[i][j]*np.cos(PA[i]-PA[j])+B[i][j]*np.sin(PA[i]-PA[j]))
            if Buses[i][7] != 2 and Buses[j][7] != 2:
                L[i-1][j-1] = V[j]*V[i]*(G[i][j]*np.sin(PA[i]-PA[j])-B[i][j]*np.cos(PA[i]-PA[j]))

for i in reversed(range(1, Buses.shape[0])):
    if Buses[i][7] == 2:
        # If Bus i is PV, remove column for that bus from N
        N = np.delete(N, i-1, 1)
        # If Bus i is PV, remove row for that bus from J
        J = np.delete(J, i-1, 0)
        # If Bus i is PV, remove row and column for that bus from L
        L = np.delete(L, i-1, 1)
        L = np.delete(L, i-1, 0)
        
display(H)
display(N)
display(J)
display(L)

array([[19.58211765, -2.42352941],
       [-2.42352941, 19.29411765]])

array([[9.69411765],
       [4.58823529]])

array([[ 2.42352941, -4.82352941]])

array([[18.35294118]])

In [9]:
Jac = np.block([[H, N], [J, L]])
Jac

array([[19.58211765, -2.42352941,  9.69411765],
       [-2.42352941, 19.29411765,  4.58823529],
       [ 2.42352941, -4.82352941, 18.35294118]])

**Compute Increment Matrix**

In [10]:
Delta_PhaseV = np.dot(inv(Jac), Delta_PQ)
Delta_Phase = Delta_PhaseV[0:len(Delta_P)]
Delta_V = Delta_PhaseV[len(Delta_P):]
display(Delta_PhaseV)
display(Delta_Phase)
display(Delta_V)

array([ 0.06942201, -0.03852157, -0.01544537])

array([ 0.06942201, -0.03852157])

array([-0.01544537])

**$\delta$ and $V$**

In [11]:
PA_new = np.zeros(Buses.shape[0])
V_new = np.zeros(Buses.shape[0])
incr = 0
for i in range(1, Buses.shape[0]):
    PA_new[i] = PA[i] + Delta_PhaseV[i-1]
    # Buses[i][7] is Bus i's BusType, 2 is PV
    if Buses[i][7] != 2:
        V_new[i] = V[i] + (Delta_V[incr]/V[i])*V[i]
        incr = incr + 1
        
PA = PA_new
V = V_new
display(PA_new)
display(V_new)

array([ 0.        ,  0.06942201, -0.03852157])

array([0.        , 0.        , 0.98455463])